# XGMIX notebook

This seems to work. One caveat is that it gets stuck in an infinte loop for the fist time you run after making changes. So just stop the kernel and re-run the cell.

In [1]:
import numpy as np
import os

In [2]:
%load_ext autoreload
%autoreload 2
from preprocess import data_process,simulate_missing_values
from XGMIX import *

## Main arguments

In [3]:
# args = {'train_data': "/home/arvindsk/XGMix/expts/xgmix_global_1/train.txt", 
#         "val_data" : "/home/arvindsk/XGMix/expts/xgmix_global_1/val.txt", 
#         "save": None,
#         "window_size": 1000,
#         "smooth_size": 75,
#         "missing":0.0}

In [4]:
args = {'train_data': "/home/arvindsk/XGMix/expts/xgmix_global_22/train.txt", 
        "val_data" : "/home/arvindsk/XGMix/expts/xgmix_global_22/val.txt", 
        "save": "/home/arvindsk/XGMix/expts/xgmix_global_22/",
        "window_size": 1000,
        "smooth_size": 75,
        "missing":0.0}

## Pre-processing

In [5]:
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)
args = Struct(**args)

win = args.window_size
training, training_labels, validation, validation_labels = data_process(args)
chmlen = training.shape[-1]
sws = args.smooth_size if args.smooth_size%2 else args.smooth_size-1
num_anc = len(np.unique(training_labels))

Loading data...
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_2//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_4//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_6//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_2//mat_map.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_4//mat_map.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_6//mat_map.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_valid/gen_2//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_valid/gen_4//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_valid/gen_6//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_v

In [6]:
if args.missing != 0:
    training = simulate_missing_values(training,args.missing)
    validation = simulate_missing_values(validation,args.missing)

In [7]:
try:
    os.mkdir(args.save)
except:
    print("Directory exists")

Directory exists


In [8]:
xgm = XGMIX(chmlen,win,sws,num_anc,args.save,cores=16)

In [9]:
xgm.train(training,training_labels,validation, validation_labels, smoothlite=10000)

Windows done: 0, 
Base Training Accuracy: 0.8233333333333334, Base Validation Accuracy: 0.71
Windows done: 100, 
Base Training Accuracy: 0.8548184818481848, Base Validation Accuracy: 0.7575247524752473
Windows done: 200, 
Base Training Accuracy: 0.8463239358761747, Base Validation Accuracy: 0.7609452736318407
Windows done: 300, 
Base Training Accuracy: 0.8485197489848653, Base Validation Accuracy: 0.7675526024363234
Smooth Training Accuracy: 1.0 
Smooth Validation Accuracy: 0.9512933753943218 


In [10]:
# predict using the built-in method.
ypreds = xgm.predict(validation)
print("Accuracy")
print(np.mean(ypreds == validation_labels.numpy()))
print("Confusion matrix")
print(sklearn.metrics.confusion_matrix(ypreds.ravel(), validation_labels.numpy().ravel()))

Accuracy
0.9512933753943218
Confusion matrix
[[27089   252    31    10     5    16    33   149]
 [   17   160     0     0     0     0     1     0]
 [   22     2 32366   139   111   283   457    34]
 [   21     2    18  8450    28     5   380  1402]
 [   30     0    44    13  8427     0    41    10]
 [    0     0     0    16     2   769     0     0]
 [    2     0    76   282    12     0 10268   495]
 [   12     0     9   124     2     0    44  2939]]


In [11]:
# predict using the standalone function.
ypreds = predict(validation,args.save+"/model.pkl")
print("Accuracy")
print(np.mean(ypreds == validation_labels.numpy()))
print("Confusion matrix")
print(sklearn.metrics.confusion_matrix(ypreds.ravel(), validation_labels.numpy().ravel()))

Accuracy
0.9512933753943218
Confusion matrix
[[27089   252    31    10     5    16    33   149]
 [   17   160     0     0     0     0     1     0]
 [   22     2 32366   139   111   283   457    34]
 [   21     2    18  8450    28     5   380  1402]
 [   30     0    44    13  8427     0    41    10]
 [    0     0     0    16     2   769     0     0]
 [    2     0    76   282    12     0 10268   495]
 [   12     0     9   124     2     0    44  2939]]
